In [536]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling as pdp
import sklearn
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import imblearn
from imblearn.over_sampling import SMOTE
from scipy.stats import pearsonr
from sklearn.feature_selection import RFE
import lightgbm as lgb 
from xgboost import XGBClassifier


In [537]:
import pandas_profiling as pdp

# from google.colab import drive
# drive.mount('/content/drive')
# %cd drive/My\ Drive/Data

In [538]:
train_data = pd.read_csv("../input/india-ml-hiring-av/train.csv")
test_data = pd.read_csv("../input/india-ml-hiring-av/test.csv")

In [ ]:
train_data.columns
train_data.profile_report()

In [ ]:
Labels = train_data['m13']

#print(Labels)
#X=train_data['interest_rate','unpaid_principal_bal','loan_term','loan_to_value','number_of_borrowers','debt_to_income_ratio','borrower_credit_score','co-borrower_credit_score','m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12']

In [ ]:
def firstpayment(train_data,k):
    firstpayment = []
    if(k==0):
        for i in range(len(train_data['origination_date'])):
            days = int(train_data['first_payment_date'][i].split('/')[0]) - int(train_data['origination_date'][i].split('-')[1])
            firstpayment.append(days)
    if(k==1):
        months = {"Feb":2,"Mar":3,"Apr":4,"May":5}
        for i in range(len(train_data['origination_date'])):
            var = months[test_X['first_payment_date'][i].split('-')[0]] - int(test_X['origination_date'][i].split('/')[1])
            firstpayment.append(var)
    train_data['firstpayment']=firstpayment
    train_data.drop(columns = ['origination_date','first_payment_date'],inplace = True)
    return train_data

In [ ]:
def mdf(k):
    train_data1 = pd.read_csv("../input/india-ml-hiring-av/train.csv")
    test_data1 = pd.read_csv("../input/india-ml-hiring-av/test.csv")
    cols=['var0','var1','var2','var3','var4','var5','var6','var7','var8','var9','var10']
    mdf=pd.DataFrame(columns=cols)
    mdf1=pd.DataFrame(columns=['val'])
    if(k==0):
        mdf1['val']=train_data1['m1']-train_data1['m12']
        cols1=['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12']
        for i in range(0,11):
            mdf[cols[i]]=train_data1[cols1[i+1]]-train_data1[cols1[i]]
    #mdf.fillna(0)
            dilmat=pd.concat([mdf1, mdf],axis=1)
            dilmat.fillna(0)
            dilmat.replace(' ',0)
    if(k==1):
        mdf1['val']=test_data1['m1']-test_data1['m12']
        cols1=['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12']
        for i in range(0,11):
            mdf[cols[i]]=test_data1[cols1[i+1]]-test_data1[cols1[i]]
    #mdf.fillna(0)
            dilmat=pd.concat([mdf1, mdf],axis=1)
            dilmat.fillna(0)
            dilmat.replace(' ',0)
    return dilmat

In [ ]:
def encode(df,columns,prefix):
    columns = columns
    df2 = pd.get_dummies(df,columns = columns, prefix = ['s','f','l'])
    #df2.drop(['source','financial_institution','loan_purpose'],axis =1)
    return df2

In [ ]:
def feature_Select(X,Labels):
    feature = []
    from scipy.stats import pearsonr
    for i in X.columns:
        correlation, _ = pearsonr(X[i],Labels)
        if abs(correlation)>0.01:
            feature.append(i)
            print(i+'\t\t\t\t '+str(correlation))
    print(feature)
   

In [ ]:
def preprocess(train_data,k):
    train_data['firstpayment'] = firstpayment(train_data,k)
#     train_data.drop(columns = ['origination_date','first_payment_date'],axis =1,inplace = True)
    train_data = encode(train_data,['source','financial_institution','loan_purpose'],['s','f','l'])
    train_data.drop('loan_id',axis=1)
    feature_Select(train_data,Labels)
    df4= train_data[['interest_rate', 'unpaid_principal_bal', 'loan_term', 'loan_to_value','debt_to_income_ratio','borrower_credit_score', 'co-borrower_credit_score','source_X','source_Y','lp_A23','lp_B12','lp_C86']]
    #mdf.drop('m13')
    dilmat=mdf(k)
    X =pd.concat([df4,dilmat], axis=1)
    sm = SMOTE(random_state=2)
    X_train,Labels = sm.fit_sample(X,Labels)
      
    return X_train
# price_standardized = standardScalar.fit(project_data['price'].values)
# this will rise the error
# ValueError: Expected 2D array, got 1D array instead: array=[725.05 213.03 329.   ... 399.   287.73   5.5 ].
# Reshape your data either using array.reshape(-1, 1)


In [ ]:
X_train = preprocess(train_data,0)
#X_train.columns
print(X_train)

#train_data.drop(['m13','loan_id','source','financial_institution','loan_purpose'],axis= 1,inplace= True)
#features= feature_Select(X,Labels)
#X_train= X[features]

X_test= preprocess(test_data,1)
#test_data.drop(['loan_id','source','financial_institution','loan_purpose'],axis= 1,inplace= True)

#X_test= Y[features]

In [ ]:
Labels1 = list(Labels)
np.bincount(Labels1)

In [ ]:

# from sklearn import svm
# wclf = svm.SVC(kernel='linear', class_weight={1:10})
# wclf.fit(X, Labels)

In [ ]:
# def model_Selection():
#     models = []
#     models.append(('LR', LogisticRegression()))
#     models.append(('LDA', LinearDiscriminantAnalysis()))
#     models.append(('KNN', KNeighborsClassifier()))
#     models.append(('CART', DecisionTreeClassifier()))
#     models.append(('NB', GaussianNB()))
#     models.append(('SVM', SVC()))
# # evaluate each model in turn
#     results = []
#     names = []
#     scoring = 'accuracy'
#     for name, model in models:
#         kfold = model_selection.KFold(n_splits=10, random_state=seed)
#         cv_results = model_selection.cross_val_score(model, X_trian,Labels, cv=kfold, scoring=scoring)
#         results.append(cv_results)
#         names.append(name)
#         msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
#         print(msg)
# # boxplot algorithm comparison
#     fig = plt.figure()
#     fig.suptitle('Algorithm Comparison')
#     ax = fig.add_subplot(111)
#     plt.boxplot(results)
#     ax.set_xticklabels(names)
#     plt.show()

In [ ]:
tdata=lgb.Dataset(X_train,label=Labels)
param = {'num_leaves':150, 'objective':'binary','max_depth':7,'learning_rate':.05,'max_bin':200}
param['metric'] = ['auc', 'binary_logloss']
num_round=50
# start=datetime.now()
lgbm=lgb.train(param,tdata,num_round)
#stop=datetime.now()
lgb_predict=lgbm.predict(X_test)
#df3.columns

In [ ]:
model = XGBClassifier(learning_rate = 0.055,max_depth = 4)
#train_x, test_x, train_Y, test_y = train_test_split(X_train,Labels,train_size=0.8)
# rfe = RFE(model, 20)
# rfe = rfe.fit(train_x,Labels)
model.fit(X_train,Labels)
# print(rfe.support_)
# print(rfe.ranking_)
xgb_predict = model.predict(X_test)
    
print( "Train Accuracy :: ", accuracy_score(train_Y, model.predict(train_x)))
print( "Test Accuracy  :: ", accuracy_score(test_y, xgb_predict))

In [ ]:
param_test6 = {
 'reg_alpha':[0.005]
}
params = {'class_weight':[{0:neg_weight, 1:1} for neg_weight in np.arange(1.0, 5.0, 0.5)]} 
# train_x, test_x, train_Y, test_y = train_test_split(X_train,Labels,train_size=0.8)
from sklearn.model_selection import GridSearchCV
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.055,max_depth=4) ,param_grid=param_test6,scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch6.fit(X_train,Labels)
gsearch6.predict(X_test)

In [ ]:

clf =RandomForestClassifier()
params = {'class_weight':[{0:neg_weight, 1:1} for neg_weight in np.arange(1.0, 5.0, 0.5)]} 
gs = GridSearchCV(estimator=clf, param_grid=params, cv=5)
gs.fit(X_train, Labels)


In [ ]:
gs.predict(X_test)

In [ ]:
def adaboost(X_train, X_test,Labels):
    model = AdaBoostClassifier(n_estimators=100, random_state=42)
    model.fit(X_train,Labels)
    y_pred = model.predict(X_test) 
    return y_pred
# AdaBoost
y_baseline = adaboost(X_train, X_test, Labels)


In [ ]:
# RUS
X_full = X_train.copy()
X_full['m13'] = Labels[1]
X_maj = X_full[X_full.target==0]
X_min = X_full[X_full.target==1]
X_maj_rus = resample(X_maj,replace=False,n_samples=len(X_min),random_state=44)
X_rus = pd.concat([X_maj_rus, X_min])
X_train_rus = X_rus.drop(['m13'], axis=1)
y_train_rus = X_rus.target
y_rus = adaboost(X_train_rus, X_test, y_train_rus)

In [ ]:
#sklearn.metrics.f1_score(Labels, X_test)
test_data1 = pd.read_csv("../input/india-ml-hiring-av/test.csv")
cols=['loan_id','m13']
output=pd.DataFrame(columns=cols)
print(output.columns)
output['loan_id']=test_data1['loan_id']
output['m13']=lgb_predict
output.to_csv("pred1.csv", index=False, header=True)